## Parte 3: Construyendo un Sistema de Recomendacion con Feedback Implicito

En este ejercicio, desarrollaremos un sistema de recomendacion con feedback implicito utilizando la libreria     [implicit](https://github.com/benfred/implicit).

**Pero, a que nos referimos con feedback implicito?**

En el primer ejercicio abordamos el filtro colaborativo el cual se basa en la suposicion de que `usuarios similares gustan de las mismas cosas/items`. La matriz usuario-item, o "matriz de utilidad" es la piedra angular del filtrado colaborativo. En la matriz de utilidad las filas representan a los usuarios y las columnas representan a los items.



Las celdas de la matriz se llenan a partir del grado de preferencia de un usuario a un item determinado y esto se representa en cualquiera de las dos formas:
1. **Feedback explicito:** feedback directo hacia un item (por ejemplo el rating de una pelicula como lo vimos en el [Ejercicio 1](https://experiencia21.tec.mx/courses/481176/assignments/15386625?module_item_id=28379086))

2. **Feedback implicito:** comportamiento indirecto hacia un item (por ejemplo el historial de compra, el historial de navegacion o historial de busquedas)

El feedback implicito hace suposiciones sobre las preferencias del usuario a partir de las acciones hacia dichos items. Si retomamos el ejemplo si miraste todos los episodios de un show y viste todas las temporadas en una semana, entonces existe la elevada posibilidad de que te guste ese show. Sin embargo, si empiezas a mirar una serie y te detienes a la mitad del primer episodio, entonces es probable que se pueda asumir que no te haya gustado ese show.



### Paso 1: Agregando las Librerias

Estos seran las librerias que utilizaremos:

- [numpy](https://numpy.org/)
- [pandas](https://pandas.pydata.org/)
- [implicit](https://github.com/benfred/implicit)
- scipy (en especifico la clase **csr_matrix**)

In [7]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

import implicit

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Paso 2: Cargando los datos

Dado que ya te has familiarizado con el dataset de MovieLens de los ejercicios 1 y 2 en este ejercicio continuaremos utilizando este dataset que puede encontrar[aqui](https://grouplens.org/datasets/movielens/), o lo puedes descargar directamente de [aqui](http://files.grouplens.org/datasets/movielens/ml-latest-small.zip). (Recuerda que estamos trabajando con los datasets `ml-latest-small.zip` )


In [8]:
ratings = pd.read_csv("datos/ratings.csv")
movies = pd.read_csv("datos/movies.csv").drop_duplicates(keep="last", subset=['title'])

Revisamos el contenido de ratings

In [9]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


En este ejercicio, definiremos el rating de las peliculas como el numero de veces que un usuario las ha mirado. Por ejemplo, si Jimena (una usuaria en nuestro dataset) le dio a la pelicula de`Batman` un rating de 1 y a `Jurassic Park` un rating de 5, podemos asumir que ha mirado la pelicula de Batman una vez y la de Jurassic Park un total de 5 veces.

### NOTA ###

Es necesario realizar una limpieza del dataset antes de proceder con el ejercicio pues contiene registros duplicados que arrojaran un problema en el numero de registros de los titulos de las peliculas. 

Para lo cual es necesario eliminar duplicados que contiene el dataset de peliculas y proceder con el ejercicio

### Paso 3: Transformando los datos

Tal y como lo hicimos en el [Ejercicio 1](https://experiencia21.tec.mx/courses/481176/assignments/15386625?module_item_id=28379086), necesitamos transformar el dataframe de `ratings` a una matriz usuario-item donde las filas representan a los usuarios y las columnas representan a las peliculas. Las celdas en esta matriz contendran el feedback implicito que en este caso es el numero de veces que un usuario ha visto una pelicula.

La funcion  `create_X()` crea una matriz de dispersion **X** con 4 diccionarios de mapeo:

- **user_mapper:** mapea user id al user index
- **movie_mapper:** mapea movie id al movie index
- **user_inv_mapper:** mapea user index al user id
- **movie_inv_mapper:** mapea movie index al movie id

Necesitamos estos diccionario por que hay que mapear las filas y columnas con la matriz de utilidad que les corresponde al user ID con su movie ID respectivamente.

Esta matriz dispersa **usuario-item** es una matriz que se obtiene al `usar scipy.sparse.csr_matrix`que almacena los datos de una manera dispersa.

In [10]:
def create_X(df: pd.DataFrame):
    """
    Generates a sparse matrix from ratings dataframe.
    
    Args:
        df: pandas dataframe
    
    Returns:
        X: sparse matrix
        user_mapper: dict that maps user id's to user indices
        user_inv_mapper: dict that maps user indices to user id's
        movie_mapper: dict that maps movie id's to movie indices
        movie_inv_mapper: dict that maps movie indices to movie id's
    """
    N = df['userId'].nunique()
    M = df['movieId'].nunique()

    user_mapper = dict(zip(np.unique(df["userId"]), list(range(N))))
    movie_mapper = dict(zip(np.unique(df["movieId"]), list(range(M))))
    
    user_inv_mapper = dict(zip(list(range(N)), np.unique(df["userId"])))
    movie_inv_mapper = dict(zip(list(range(M)), np.unique(df["movieId"])))
    
    user_index = [user_mapper[i] for i in df['userId']]
    movie_index = [movie_mapper[i] for i in df['movieId']]

    X = csr_matrix((df["rating"], (movie_index, user_index)), shape=(M, N))
    
    return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper

In [11]:
X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper = create_X(ratings)

### Creando los Mapeos de los titulos de las peliculas

Necesitamos traducir el titulo de una pelicula a partir de su indice en la matriz usuario-item y vice versa. Vamos a crear dos funciones que nos ayuden con esta traduccion.

- `get_movie_index()` - convierte el titulo de una pelicula a su indice. Hace uso de la funcion de comparacion de strings que se le pasan a [fuzzywuzzy](https://github.com/seatgeek/fuzzywuzzy) 
 para obtener el titulo de una pelicula que se le pase. Esto significa que no necesitamos saber la forma de escribir o el formato de una pelicula para obtener su indice.

- `get_movie_title()` - convierte el indice de una pelicula a su titulo.

In [12]:
from fuzzywuzzy import process

def movie_finder(title):
    all_titles = movies['title'].tolist()
    closest_match = process.extractOne(title,all_titles)
    return closest_match[0]

movie_title_mapper = dict(zip(movies['title'], movies['movieId']))
movie_title_inv_mapper = dict(zip(movies['movieId'], movies['title']))

def get_movie_index(title):
    fuzzy_title = movie_finder(title)
    movie_id = movie_title_mapper[fuzzy_title]
    movie_idx = movie_mapper[movie_id]
    return movie_idx

def get_movie_title(movie_idx): 
    movie_id = movie_inv_mapper[movie_idx]
    title = movie_title_inv_mapper[movie_id]
    return title 

/Users/ricardo/Documents/Maestria/BigData/Parte 3/venv/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Vamos a probar esta funcion para obtener el indice de `Jurassic Park`. 

In [13]:
get_movie_index('Jurasc prk')

418

Utilizemos el indice obtenido con la funcion `get_movie_title()`. Tendremos que obtener el titulo de Jurassic Park.

In [14]:
get_movie_title(418)

'Jurassic Park (1993)'

Con esto podemos comprobar que las funciones nos permitiran interpretar las recomendaciones obtenidas del sistema.

### Paso 4: Construyendo el modelo de modelo de Recomendacion de Feedback Implicito

Una vez que hemos transformado nuestros datos ahora si podemos empezar a construir nuestro modelo de recomendacion.


La libreria [implicit](https://github.com/benfred/implicit) esta basada en un factorizacion de matrices (tomado del algebra lineal). Esto nos permite hallar caracteristicas
latentes que se esconden en las interacciones entre los usuarios y las peliculas. Estas caracteristicas latentes nos brindan una representacion mas compacta de los gustos
de los usuarios y la descripcion de un item. La factorizacion matricial es particularmente util para datos muy dispersos y puede mejorar la calidad de las recomendaciones
obtenidas. El algoritmo opera al factorizar la matris usuario-item en dos matrices:

- matriz usuario-factorers  (n_users, k)
- matriz item-factorers     (k, n_items)

Reduciremos las dimensiones de nuestra matriz original a nuestras dimensiones particulares. No es posible interpretar cada caracteristica latente $k$. Sin embargo,
podemos suponer que una caracteristica latente puede representar a los usuarios que gusten de comedia romantica de los 90s, mientras que otra caracteristica lantente
puede representar a peliculas independientes extranjeras.


$$X_{mn} \approx P_{mk} \times Q_{nk}^T = \hat{X}$$



En el caso de una factorizacion matricial tradicional como [SVD](https://www.freecodecamp.org/news/singular-value-decomposition-vs-matrix-factorization-in-recommender-systems-b1e99bc73599/) lo que hariamos seria intentar resolver la factorizacion de una sola vez, sin embargo esto resultaria muy costoso computacionalmente. Otra forma de atacar este problem es utilizando una tecnica denominada
[Minimos Cuadrados Alternos, Alternating Least Squares (ALS)](https://sophwats.github.io/2018-04-05-gentle-als.html). Ocupando ALS, podemos resolver una matriz de factores a la vez:

- Paso 1: Fijamos la matriz de factores de usuario (user-factor) y resolvemos la matriz de factores de elementos (item-factor)
- Paso 2: Fijamos la matriz de factores de elementos (item-factor) y resolvemos la matriz de factores de usuario (user-factor)

Al alternar los pasos 1 y 2 hasta que el producto punto de la matriz de factores de elementos (item-factor) y la matriz de factores de usuarios (user-item) es aproximadamente igual a la matrix original X (user-item). Este procedimiento es comptacionalmente menos costoso y puede ser parelelizado.

La libreria `implicit` implementa una factorizacion matricial utilizando ALS (puedes consultar los detalles [aqui](https://implicit.readthedocs.io/en/latest/als.html))

In [15]:
model = implicit.als.AlternatingLeastSquares(factors=50)


/Users/ricardo/Documents/Maestria/BigData/Parte 3/venv/lib/python3.10/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 10 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


Este modelo viene con algunos hyperparametros que deben ser ajustados para generar resultados optimos:

- los factores ($k$): numero de factores latentes,
- regularizacion ($\lambda$): evita que el modelo caiga en overfitting durante el entrenamiento

Para este ejercicio definiremos $k = 50$ y $\lambda = 0.01$ como los valores a utilizar. 

El siguiente paso ahora es ajustar nuestro modelo a la matriz user-item.


In [16]:
model.fit(X.T.tocsr())

100%|██████████| 15/15 [00:01<00:00, 10.93it/s]



Ahora pongamos a prueba las recomendaciones de nuestro modelo. Podemos utilizar el metodo `similar_items()` que nos muestra las peliculas mas relevantes dada una pelicula en especifico. De igual forma, podemos utilizar la funcion `get_movie_index()` para obtener el indice de la pelicula si es que es una pelicula que nos interesa a partir de las recomendaciones obtenidas.

In [17]:
movie_of_interest = 'forrest gump'

movie_index = get_movie_index(movie_of_interest)
related = model.similar_items(movie_index)
related

(array([ 314,  277,  510,  257,  461,   97,  123,  418, 1938,   43],
       dtype=int32),
 array([1.0000001 , 0.88227344, 0.8449495 , 0.8425889 , 0.72428703,
        0.71467316, 0.68267596, 0.6766566 , 0.6451257 , 0.6318825 ],
       dtype=float32))

Lo que obtenemos de `similar_items()` no es facil de leer por lo que necesitamos de la funcion `get_movie_title()` para interpretar los resultados.

In [18]:
print(f"Por que miraste la pelicula de {movie_finder(movie_of_interest)} te pueden interesar las siguientes peliculas:")
for t, r in zip(related[0], related[1]):
    
    recommended_title = get_movie_title(t)
    if recommended_title != movie_finder(movie_of_interest):
        print(recommended_title)



Por que miraste la pelicula de Forrest Gump (1994) te pueden interesar las siguientes peliculas:
Shawshank Redemption, The (1994)
Silence of the Lambs, The (1991)
Pulp Fiction (1994)
Schindler's List (1993)
Braveheart (1995)
Apollo 13 (1995)
Jurassic Park (1993)
Matrix, The (1999)
Seven (a.k.a. Se7en) (1995)


Al usar el rating de los usuarios como feedback implicito, los resultados se ven bien. Intenta cambiando la variable `movie_of_interest`.

### Paso 5: Generando las recomendaciones del usuario

Una caracteristica interesante de `implicit` es que puedes obtener recomendaciones personalizadas para un usuario determinado. Intentemos ver los resultados con un usuario especifico de nuestro dataset.

In [19]:
user_id = 90

In [20]:
user_ratings = ratings[ratings['userId']==user_id].merge(movies[['movieId', 'title']])
user_ratings = user_ratings.sort_values('rating', ascending=False)
print(f"El numero de peliculas rankeadas por el usuario {user_id} es de: {user_ratings['movieId'].nunique()}")

El numero de peliculas rankeadas por el usuario 90 es de: 54


En este caso vemos que el usuario 90 miro 54 peliculas y el rating de su favoritas son:

In [21]:
user_ratings = ratings[ratings['userId']==user_id].merge(movies[['movieId', 'title']])
user_ratings = user_ratings.sort_values('rating', ascending=False)
top_5 = user_ratings.head()
top_5

,userId,movieId,rating,timestamp,title
49,90,1411,5.0,856354329,Hamlet (1996)
12,90,85,5.0,856354195,Angels and Insects (1995)
46,90,1357,5.0,856354301,Shine (1996)
30,90,818,5.0,856354233,"Very Brady Sequel, A (1996)"
47,90,1358,5.0,856354551,Sling Blade (1996)


Las peliculas con el menor rating son:

In [22]:
bottom_5 = user_ratings[user_ratings['rating']<5].tail()
bottom_5

,userId,movieId,rating,timestamp,title
21,90,685,3.0,856354348,It's My Party (1996)
14,90,141,3.0,856353996,"Birdcage, The (1996)"
10,90,81,3.0,856354174,Things to Do in Denver When You're Dead (1995)
9,90,68,3.0,856354249,French Twist (Gazon maudit) (1995)
53,90,1442,2.0,856354612,Prefontaine (1997)


A partir de las preferencias anteriores, podemos inferir algo acerca del usuario 90. Veamos que recomendaciones se pueden generar para este usuario en particular.

Utilizaremos `recommend()` que utiliza el indice del usuario y lo transpone con la matriz user-item.

In [23]:
X_t = X.T.tocsr()
user_idx = user_mapper[user_id]
recommendations = model.recommend(user_idx, X_t[user_idx])
recommendations

(array([ 546,  594,   55,  621, 1072,  533,  910,  592,   84,  658],
       dtype=int32),
 array([0.8126131 , 0.77031416, 0.7172132 , 0.7026186 , 0.7001592 ,
        0.675643  , 0.65410584, 0.64814883, 0.62109774, 0.6012687 ],
       dtype=float32))

No podemos interpretar los resultados obtenidos pues estan listados los indices. Hagamos una conversion del indice al titulo de las peliculas recomendadas.

In [24]:
for t, r in zip(recommendations[0], recommendations[1]):
    recommended_title = get_movie_title(t)
    print(recommended_title)

Mission: Impossible (1996)
Twister (1996)
Mr. Holland's Opus (1995)
Eraser (1996)
Jerry Maguire (1996)
Primal Fear (1996)
Star Wars: Episode VI - Return of the Jedi (1983)
Rock, The (1996)
Broken Arrow (1996)
Tin Cup (1996)


Que podemos decir acerca de las recomendaciones para este usuario?

Intentemos con otro usuario y analicemos los resultados obtenidos.